In [150]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as np

In [151]:
import requests

In [155]:
def pri(i):
    l = []
    url= "https://www.zomato.com/bangalore/south-bangalore-restaurants?page="+str(i)
    SB_restaurants = {}
    restaurant_id = 1000
    def extract_source(url):
            headers = {"User-Agent":"Mozilla/5.0"}
            source=requests.get(url, headers=headers).text
            return source

    soup = BeautifulSoup(extract_source(url),"html.parser")
    restaurants = soup.find_all('article',{'class':'search-result'})
    j = []
    for restaurant in restaurants :
        total = restaurant.find('div',{'class':'ta-right'}).text
        j.append(total)
    f = []
    for i in j:
        if(i[-11]=='\n'):
            f.append(i[-10:-7])
        else:
            f.append(i[-11:-7])
            
    f = pd.DataFrame(f,columns=['Votes'])
    
    for restaurant in restaurants :      
        restaurant_name = restaurant.find('a',{'class':'result-title'}).text
        restaurant_tag = restaurant.find('div',{'class':'res-snippet-small-establishment'})
        restaurant_type = restaurant_tag.text if restaurant_tag else 'N/A'
        area = restaurant.find('b').text
        ratings = restaurant.find('div',{'class':'rating-popup'}).text
        restaurant_id += 1
        l.append([restaurant_id ,restaurant_name ,ratings,area])
    return l,f

In [158]:
k,f = pri(1)

In [165]:
import pandas as pd
h = pd.DataFrame(k,columns = ['Restaurant_ID','Restaurant_name','Restaurant_Rating','Area'])
h['Votes'] = f
for i in range(1,6):
    k,f = pri(i+1)
    l = pd.DataFrame(k,columns = ['Restaurant_ID','Restaurant_name','Restaurant_Rating','Area'])
    l['Restaurant_ID'] = l.iloc[l.index]['Restaurant_ID']+i*15
    l['Votes'] = f
    h = h.append(l)

In [166]:
h.index = h.Restaurant_ID
h = h.drop('Restaurant_ID',axis=1)

In [167]:
h['Restaurant_Rating'] = h['Restaurant_Rating'].apply(lambda x: x[41:44])

In [187]:
h = h.drop([1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088,
            1089, 1090],axis=0)

In [189]:
h.to_csv('abhijeet_bhatt.csv')